
## Project Phase 2 - Shreya Mishra


Install amplpy, pandas and other packages.

In [1]:
!pip install -q amplpy ampltools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 31.6 MB/s eta 0:00:00


Setup AMPL and select solvers.

In [2]:
# Google Colab & AMPL integration
MODULES, LICENSE_UUID = ["coin", 'gurobi', "cplex", "highs", "gokestrel"], "42fc7eb6-69aa-445d-b655-3ad24d836541"
from amplpy import tools
from ampltools import cloud_platform_name, ampl_notebook, register_magics

# instantiate AMPL object and register magics
if cloud_platform_name() is None:
    ampl = AMPL() # Use local installation of AMPL
else:
    ampl = tools.ampl_notebook(modules=MODULES, license_uuid=LICENSE_UUID, g=globals())

register_magics(ampl_object=ampl)

Licensed to Bundle #6741.7193 expiring 20241231: INFO 645 Prescriptive Analytics, Prof. Paul Brooks, Virginia Commonwealth University.


Mount Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Read data

In [48]:
import pandas as pd
from amplpy import AMPL

def excel_to_dict(file_path):
    # Read the Excel file
    excel_data = pd.ExcelFile(file_path)

    # Create a dictionary to store each sheet as a DataFrame
    sheet_dict = {}

    # Loop through each sheet and convert it to a dictionary
    for sheet_name in excel_data.sheet_names:
        # Read the sheet into a DataFrame
        df = pd.read_excel(excel_data, sheet_name=sheet_name)

        # Convert the DataFrame to a dictionary where columns are keys
        sheet_dict[sheet_name] = df

    return sheet_dict

# Path to your Excel file
file_path = '/content/drive/MyDrive/realignment_data_v1.2.xlsx'

# Read all sheets in the workbook and convert them to DataFrames
sheets_data = excel_to_dict(file_path)

# Extract data from the sheets for use in the model
regional_office_data = sheets_data['Regional Office Data']
hours_required_data = sheets_data['Hours Required']
travel_cost_data = sheets_data['Mileage']
travel_time_data = sheets_data['Travel Time']
annual_trips_data = sheets_data['Annual Trips Required']

# Rename 'Unnamed: 0' to 'Store' or 'Office' for easier reference
hours_required_data.rename(columns={'Unnamed: 0': 'Store'}, inplace=True)
annual_trips_data.rename(columns={'Unnamed: 0': 'Store'}, inplace=True)
regional_office_data.rename(columns={'Unnamed: 0': 'Office'}, inplace=True)
travel_cost_data.rename(columns={'Unnamed: 0': 'Store'}, inplace=True)
travel_time_data.rename(columns={'Unnamed: 0': 'Store'}, inplace=True)

In [49]:
# Initialize AMPL object
ampl = AMPL()


Define model.

In [76]:
ampl.eval('''
reset;

# Sets for the problem
set STORES;
set OFFICES;
set AREAS := 1..5;

# Parameters
param hours_required {STORES, AREAS};
param hours_available {OFFICES, AREAS};
param mileage {STORES, OFFICES};
param travel_time {STORES, OFFICES};
param annual_trips {STORES, AREAS};
param mileage_rate default 0.585;
param salary_rate default 26;

# Decision variable: binary assignment of stores to offices
var Assign {STORES, OFFICES} binary;

# Calculate Mileage Cost and Salary Cost
param mileage_cost {s in STORES, o in OFFICES} := mileage[s, o] * mileage_rate;
param salary_cost {s in STORES, o in OFFICES} := travel_time[s, o] * salary_rate;

# Total Cost for each store-office combination
param total_cost {s in STORES, o in OFFICES} := mileage_cost[s, o] + salary_cost[s, o];

# Objective function: minimize total cost

minimize Total_Cost:
    sum {s in STORES, o in OFFICES, a in AREAS} annual_trips[s, a] * 2 * total_cost[s, o] * Assign[s, o];


# Constraint: each store must be assigned to exactly one office
subject to One_Office_Per_Store {s in STORES}:
    sum {o in OFFICES} Assign[s, o] = 1;

# Constraint: each office must have enough available hours in each area to meet store requirements
subject to Area_Availability {o in OFFICES, a in AREAS}:
    sum {s in STORES} hours_required[s, a] * Assign[s, o] <= hours_available[o, a];

# Constraint: limit total resource capacity (hours required + travel time) at each office


subject to Resource_Capacity {o in OFFICES, a in AREAS}:
    sum {s in STORES} (hours_required[s, a] + (travel_time[s, o] * 2 * annual_trips[s, a])) * Assign[s, o] <= hours_available[o, a];

''')


Provide data to the model.

In [77]:
# Load data into AMPL

# Load stores and offices into sets
ampl.set['STORES'] = hours_required_data['Store'].tolist()
ampl.set['OFFICES'] = regional_office_data['Office'].tolist()

#ampl.param['hours_required'] = hours_required
#ampl.param['hours_available'] = hours_available
#ampl.param['mileage'] = mileage
#ampl.param['travel_time'] = travel_time
#ampl.param['annual_trips'] = annual_trips
#ampl.param['mileage_rate'] = mileage_rate
#ampl.param['salary_rate'] = salary_rate

mileage_rate = 0.585
salary_rate = 26

ampl.param['mileage_rate'] = mileage_rate
ampl.param['salary_rate'] = salary_rate

# Load parameters
# Load hours required for each store and area
hours_required = {}
for _, row in hours_required_data.iterrows():
    store = row['Store']
    for area, column in enumerate(['Inventory', 'Payroll', 'Hiring', 'Marketing', 'Merchandising'], start=1):
        hours_required[store, area] = row[column]
ampl.param['hours_required'] = hours_required

# Load hours available for each office and area
hours_available = {}
for _, row in regional_office_data.iterrows():
    office = row['Office']
    for area, column in enumerate(['Inventory', 'Payroll', 'Hiring', 'Marketing', 'Merchandising'], start=1):
        hours_available[office, area] = row[column]
ampl.param['hours_available'] = hours_available

# Load travel cost (mileage) and time data
mileage = {}
for _, row in travel_cost_data.iterrows():
    store = row['Store']
    for office in ['Staunton', 'Warrenton', 'Richmond', 'Tappahannock']:
        if row[office] != '-':
            mileage[store, office] = row[office]
ampl.param['mileage'] = mileage

travel_time = {}
for _, row in travel_time_data.iterrows():
    store = row['Store']
    for office in ['Staunton', 'Warrenton', 'Richmond', 'Tappahannock']:
        if row[office] != '-':
            travel_time[store, office] = row[office]
ampl.param['travel_time'] = travel_time

# Load annual trips for each store and area
annual_trips = {}
for _, row in annual_trips_data.iterrows():
    store = row['Store']
    for area, column in enumerate(['Inventory', 'Payroll', 'Hiring', 'Marketing', 'Merchandising'], start=1):
        annual_trips[store, area] = row[column]
ampl.param['annual_trips'] = annual_trips


Display problem formulation.

In [ ]:
#ampl.eval('''expand;''')

Set solver and solve.

In [78]:
# Set the solver to CPLEX
ampl.setOption('solver', 'cplex')

# Solve the model
ampl.solve()

CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 195479.31
17 simplex iterations
absmipgap=2.91038e-11, relmipgap=0


Print solution and results.

In [79]:
# Print the total cost after solving the model
total_cost = ampl.get_objective('Total_Cost').value()
print(f'Total Cost: {total_cost}')

# Get the results
assign = ampl.get_variable('Assign').get_values()
#print(assign)

# Convert the assignment results to a DataFrame for better readability
assign_df = assign.to_pandas()
print(assign_df)  # Print assignment decisions in tabular form

# The model will assign stores to regional offices to minimize the cost, respecting capacity constraints.

Total Cost: 195479.30999999997
                                  Assign.val
index0              index1                  
Albemarle_County    Richmond               0
                    Staunton               1
                    Tappahannock           0
                    Warrenton              0
Amherst_County      Richmond               0
...                                      ...
Westmoreland_County Warrenton              0
York_County         Richmond               0
                    Staunton               0
                    Tappahannock           1
                    Warrenton              0

[172 rows x 1 columns]


In [80]:
# Filter DataFrame to show only the assignments made (Assign.val == 1)
assigned_pairs_df = assign_df[assign_df['Assign.val'] == 1]
print(assigned_pairs_df)  # Print only the rows with assignments


                                     Assign.val
index0                 index1                  
Albemarle_County       Staunton               1
Amherst_County         Staunton               1
Augusta_County         Staunton               1
Buckingham_County      Staunton               1
Caroline_County        Tappahannock           1
Charles_City_County    Richmond               1
Chesterfield_County    Richmond               1
City_of_Fredericksburg Tappahannock           1
City_of_Richmond       Richmond               1
Culpeper_County        Warrenton              1
Cumberland_County      Richmond               1
Dinwiddie_County       Richmond               1
Essex_County           Tappahannock           1
Fauquier_County        Warrenton              1
Fluvanna_County        Richmond               1
Goochland_County       Richmond               1
Greene_County          Staunton               1
Hanover_County         Richmond               1
Henrico_County         Richmond         